In [ ]:
import numpy as numpy
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])

# Converting Images into the tensor, and normalize it for every image

In [ ]:
train_data = torchvision.datasets.CIFAR10(root='/home/akash-1/nn_scratch',train=True,transform= transform,download=True)
test_data =  torchvision.datasets.CIFAR10(root='/home/akash-1/nn_scratch',train=False,transform= transform,download=True)

train_loader = torch.utils.data.DataLoader(train_data,batch_size=32,shuffle=True,num_workers=2)
test_loader =torch.utils.data.DataLoader(test_data,batch_size=32,shuffle=True,num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
image,label = train_data[0]

In [ ]:
image.size() # 3channel ,

torch.Size([3, 32, 32])

In [ ]:
class_name = ['plane ','car','bird','cat','deer','dog','frog','horse','ship','truck']

In [ ]:
class NeuralNet(nn.Module):

    def __init__(self):
        super().__init__() #inherit all functions of nn.module class

        self.conv1 = nn.Conv2d(3,12,5,stride=1,padding=0)
        self.pool = nn.MaxPool2d((2,2),padding=0) #play around with average pooling as well (12,14,14)
        self.conv2 = nn.Conv2d(12,24,5,stride=1,padding=0) #(10,10), 24 filters, use a max pool again

        self.fc1 = nn.Linear(24*5*5,120) #flatenns and inputs it to 120 neurons in input
        self.fc2 =nn.Linear(120,84)#124 neurons fully connected to 84 neurons
        self.fc3 = nn.Linear(84,10)


    def forward(self,x):

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x,1)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)


        return x














In [ ]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss() #negative log loss
optimizer = optim.SGD(net.parameters(),lr=0.01,momentum =0.9)

for layer in net.modules():  #using xavier initialization
    if isinstance(layer,nn.Linear):
        nn.init.xavier_normal_(layer.weight)
        nn.init.zeros_(layer.bias)


In [ ]:

for epoch in range(1, 31):                       # 1..30 instead of 0..29
    print(f'Training Epoch {epoch}..')
    running_loss = 0.0

    for i, data in enumerate(train_loader, start=1):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 100 == 0:
            print(f'Epoch {epoch} Batch {i} AvgLoss: {running_loss / i:.4f}')

    print(f'Epoch {epoch} finished. AvgLoss: {running_loss / len(train_loader):.4f}')









Training Epoch 1..
Epoch 1 Batch 100 AvgLoss: 2.1701
Epoch 1 Batch 200 AvgLoss: 2.0321
Epoch 1 Batch 300 AvgLoss: 1.9484
Epoch 1 Batch 400 AvgLoss: 1.8863
Epoch 1 Batch 500 AvgLoss: 1.8323
Epoch 1 Batch 600 AvgLoss: 1.7917
Epoch 1 Batch 700 AvgLoss: 1.7550
Epoch 1 Batch 800 AvgLoss: 1.7256
Epoch 1 Batch 900 AvgLoss: 1.6994
Epoch 1 Batch 1000 AvgLoss: 1.6748
Epoch 1 Batch 1100 AvgLoss: 1.6558
Epoch 1 Batch 1200 AvgLoss: 1.6359
Epoch 1 Batch 1300 AvgLoss: 1.6167
Epoch 1 Batch 1400 AvgLoss: 1.6027
Epoch 1 Batch 1500 AvgLoss: 1.5888
Epoch 1 finished. AvgLoss: 1.5797
Training Epoch 2..
Epoch 2 Batch 100 AvgLoss: 1.3248
Epoch 2 Batch 200 AvgLoss: 1.3035
Epoch 2 Batch 300 AvgLoss: 1.3088
Epoch 2 Batch 400 AvgLoss: 1.3055
Epoch 2 Batch 500 AvgLoss: 1.2996
Epoch 2 Batch 600 AvgLoss: 1.2923
Epoch 2 Batch 700 AvgLoss: 1.2884
Epoch 2 Batch 800 AvgLoss: 1.2865
Epoch 2 Batch 900 AvgLoss: 1.2834
Epoch 2 Batch 1000 AvgLoss: 1.2775
Epoch 2 Batch 1100 AvgLoss: 1.2727
Epoch 2 Batch 1200 AvgLoss: 1.2654
E

In [ ]:
torch.save(net.state_dict(),'tarined_net.pth')

In [ ]:
net = NeuralNet()
net.load_state_dict(torch.load('tarined_net.pth'))

/tmp/ipykernel_25802/1457915640.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('tarined_net.pth'))


<All keys matched successfully>

In [ ]:
correct =0
total =0

net.eval()

with torch.no_grad():

    for data in test_loader:
        image ,labels = data

        output = net(image)
        _,predicted = torch.max(output,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()




accuracy = 100 *correct /total

print(f'accuracy: {accuracy}%')

accuracy: 64.14%


In [ ]:
new_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))

])

def load_image(image_path):

    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0) #presents a single image a batch

    return image

image_paths = [
    '/home/akash-1/nn_scratch/airbus_a380_on_the_ground.jpg',
    '/home/akash-1/nn_scratch/Untitled.jpeg'
]
images = [load_image(img)for img in image_paths]

net.eval()
with torch.no_grad():
    for image in images:
        output = net(image)
        _, predicted = torch.max(output,1)

        print(f'prediction: {class_name[predicted.item()]}')








prediction: plane 
prediction: car


In [ ]:
net=NeuralNet()

image_paths